<a href="https://colab.research.google.com/github/Bobisreallyme/TestRepo/blob/main/Cleaned_Urlscraper_and_databasegenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!pip install webdriver_manager
!pip install odfpy
!pip install google-colab-selenium
!pip install chat_downloader

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.edge.service import Service
import time
import numpy as np
from datetime import datetime
import pandas as pd
import google_colab_selenium as gs
from googleapiclient.discovery import build

from google.colab import drive
drive.mount('/content/drive')

You will need to provide the link to your ods file containing the urls/database names and locations you need to create, as well as a youtube api key so we can extract the dates and times.

In [ ]:
ODS_FILE_PATH = 'ODS_File.ods'
API_KEY_INIT = 'Your API Key'

The code below is for generating the list of urls of livestreams produced by a channel, along with a list of dates/times/titles

In [ ]:
#GET THE ODS FILE CONTAINING THE URLS AND THE LOCATIONS + NAMES OF THE ODS OF URLS YOU WANT TO CREATE
data_frame_channels = pd.read_excel(ODS_FILE_PATH)
channel_url_all = data_frame_channels.iloc[:,0].tolist()
spreadsheet_name_all = data_frame_channels.iloc[:,1].tolist()

#FUNCTION TO SCRAP THE CHANNEL URL
def get_livestreamurls_fromchannel(channel_url):
    #INITIALIZE WEBDRIVER
    driver = gs.Chrome()
    #OPEN CHANNEL URL
    driver.get(channel_url)

    #LOAD THE VIDEOS, BY SCROLLING TO THE BOTTOM OF THE PAGE
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(2)
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    #WAIT FOR LIVESTREAM URLS TO LOAD
    wait = WebDriverWait(driver, 10)
    videos = wait.until(EC.presence_of_all_elements_located(
        (By.CSS_SELECTOR, 'a.yt-simple-endpoint.style-scope.ytd-rich-grid-media')
    ))

    #EXTRACT AND PRINT URLS
    urls = [video.get_attribute('href') for video in videos]

    #APPEND THE URLS
    urls_all = []
    for url in urls:
        if not not url:
            urls_all.append(url)
            print(url)

    #CLOSE BROWSER
    driver.quit()
    return urls_all


#USE THE GOOGLE API TO GET DATES/TIMES/TITLES OFF YOUTUBE
def API_get_title_date(url):
    API_KEY = API_KEY_INIT
    youtube = build('youtube', 'v3', developerKey=API_KEY)
    temp_ids = url.split('https://www.youtube.com/watch?v=')[1]
    temp_ids = temp_ids.split('&t=')[0]
    video_id = temp_ids
    video_response = youtube.videos().list(
        part='snippet',
        id=video_id
    ).execute()

    upload_date_str = video_response['items'][0]['snippet']['publishedAt']
    upload_date = datetime.strptime(upload_date_str, "%Y-%m-%dT%H:%M:%SZ")
    video_name = video_response['items'][0]['snippet']['title']
    video_status = video_response['items'][0]['snippet']['liveBroadcastContent']

    return video_name,upload_date,video_status


#LOOP TO EXTRACT THE DATE GIVEN THE ABOVE FUNCTIONS AND WRITE THEM INTO AN ODS FILE. IF ANY CANNOT BE FETCHED, NOTE THAT IT HAS BEEN EXCLUDED
for index_temp in range(0,len(channel_url_all)):
    channel_url = channel_url_all[index_temp]
    channel_url
    spreadsheet_name = spreadsheet_name_all[index_temp]
    spreadsheet_name

    urls_all = get_livestreamurls_fromchannel(channel_url)


    video_name_all = []
    upload_date_all = []
    video_url_all = []


    for urls_temp in urls_all:
        video_name,upload_date,video_status = API_get_title_date(urls_temp)
        if not video_status == 'upcoming':
            video_name_all.append(video_name)
            upload_date_all.append(upload_date)
            video_url_all.append(urls_temp)
        else:
            print('excluded')
            print(urls_temp)


    temp_sort_order = np.argsort(upload_date_all)


    sorted_dates_all = [upload_date_all[index].strftime("%Y-%m-%dT%H:%M:%SZ") for index in temp_sort_order]
    sorted_titles_all = [video_name_all[index] for index in temp_sort_order]
    sorted_urls_all = [video_url_all[index] for index in temp_sort_order]


    aggregated_streams = {}
    aggregated_streams['titles'] = sorted_titles_all
    aggregated_streams['urls'] = sorted_urls_all
    aggregated_streams['dates'] = sorted_dates_all


    dataframe_aggregated_streams = pd.DataFrame(aggregated_streams)


    with pd.ExcelWriter(spreadsheet_name,engine="odf") as writer:
        dataframe_aggregated_streams.to_excel(writer)

Given the generated urls, extract the livestream chat data and store as an SQLite3 database of the form:

id/URL/TypeofMessages/ListofTimes/ListofBadges/ListofAuthors/ListofMessages/DonationAmount/DonationCurrency

Extraction is done using the chat_downloader library developed by xenova (Joshua Lochner)

In [ ]:
from chat_downloader import ChatDownloader

In [ ]:
#CHANGE THESE ACCORDING TO HOW YOU WANT TO NAME THE DATABASE
spreadsheet_name = r"url_list.ods"
db_document = r'Db_file.dbs'

Alldataframe = pd.read_excel(spreadsheet_name)

ListofStreamUrls = Alldataframe.iloc[:,2].tolist()

In [ ]:
Allchat = []

def GetChatfromstream(urlinput):
    try:
        return ChatDownloader().get_chat(urlinput, message_groups=['messages', 'superchat'])
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return []



import sqlite3
conn = sqlite3.connect(db_document)
batchsize = 100
c = conn.cursor()
c.execute('''CREATE TABLE IF NOT EXISTS DBdb (
                        id INTEGER PRIMARY KEY,
                        URL TEXT,
                        TypeofMessages TEXT,
                        ListofTimes TEXT,
                        ListofBadges TEXT,
                        ListofAuthors TEXT,
                        ListofMessages TEXT,
                        DonationAmount TEXT,
                        DonationCurrency TEXT
                        )''')


i = 1
j = 1
for URL in ListofStreamUrls:

    chat = GetChatfromstream(URL)

    batch = []
    print(j)
    if chat!=[]:
        for message in chat:
            try:
                test = message['time_in_seconds']
                if message['message_type'] == 'paid_message':
                    ListofMessages = message['message']
                    DonationAmount = message['money']['amount']
                    DonationCurrency = message['money']['currency']
                elif message['message_type'] == 'paid_sticker':
                    ListofMessages = 'NULL'
                    DonationAmount = message['money']['amount']
                    DonationCurrency = message['money']['currency']
                else:
                    ListofMessages = message['message']
                    DonationAmount = 'NULL'
                    DonationCurrency = 'NULL'

                ListofAuthors = message['author']['name']

                if 'badges' not in message['author']:
                    ListofBadges = 'NULL'
                else:
                    ListofBadges = message['author']['badges'][0]['title']

                ListofTimes = message['time_in_seconds']
                TypeofMessages = message['message_type']

            except KeyError:

                print('KeyError Encountered')
                break

            batch.append((i,URL,TypeofMessages,ListofTimes,ListofBadges,ListofAuthors,ListofMessages,DonationAmount,DonationCurrency))
            i=i+1

            if len(batch) >= batchsize:
                c.executemany('''INSERT INTO DBdb
                            (id, URL, TypeofMessages, ListofTimes, ListofBadges, ListofAuthors, ListofMessages, DonationAmount, DonationCurrency)
                            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)''', batch)
                batch = []

        if batch:
            c.executemany('''INSERT INTO DBdb
                            (id, URL, TypeofMessages, ListofTimes, ListofBadges, ListofAuthors, ListofMessages, DonationAmount, DonationCurrency)
                            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)''', batch)
        j = j + 1
        conn.commit()

    print('save completed')